In [ ]:
# ======================================================
# Exploratory Data Analysis on GOLD LAYER - Jupyter (Pandas)
# Dataset: abc_dw_gl_pr_po_kpi.csv
# ======================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

pd.set_option("display.max_columns", None)

# ------------------------------------------------------
# 1. LOAD GOLD DATASET
# ------------------------------------------------------
# Make sure the CSV is in the same folder as this notebook
file_path = "abc_dw_gl_pr_po_kpi.csv"

df = pd.read_csv(file_path)

print("Record Count:", df.shape[0])
print("Column Count:", df.shape[1])
display(df.head())

# ------------------------------------------------------
# 2. SCHEMA REVIEW
# ------------------------------------------------------
print("\nData types (schema):")
df.info()

# ------------------------------------------------------
# 3. SUMMARY STATISTICS (NUMERIC KPIs)
# ------------------------------------------------------
numeric_cols = [
    "pr_approval_ageing",
    "po_approval_ageing",
    "pr_to_po_ageing"
]

print("\nSummary statistics for key numeric KPIs:")
display(df[numeric_cols].describe())

# ------------------------------------------------------
# 4. MISSING VALUE CHECK
# ------------------------------------------------------
print("\nMissing values (count):")
missing_count = df.isna().sum()
display(missing_count.to_frame("missing_count"))

print("\nMissing values (percentage):")
missing_pct = (df.isna().sum() / len(df)) * 100
display(missing_pct.to_frame("missing_pct"))

# ------------------------------------------------------
# 5. DISTRIBUTION OF KEY KPIs
# ------------------------------------------------------
for col in numeric_cols:
    plt.figure(figsize=(6,4))
    sns.histplot(df[col].dropna(), kde=True, bins=30)
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()

# ------------------------------------------------------
# 6. CORRELATION ANALYSIS (NUMERIC KPIs)
# ------------------------------------------------------
print("\nCorrelation matrix for numeric KPIs:")
corr_matrix = df[numeric_cols].corr()
display(corr_matrix)

plt.figure(figsize=(6,4))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix - Key KPIs")
plt.show()

# ------------------------------------------------------
# 7. PR & PO TREND ANALYSIS
# ------------------------------------------------------
# Convert date-like columns to datetime if they exist
if "pr_creationdate" in df.columns:
    df["pr_creationdate"] = pd.to_datetime(df["pr_creationdate"], errors="coerce")

if "po_createdon" in df.columns:
    df["po_createdon"] = pd.to_datetime(df["po_createdon"], errors="coerce")

# PR Volume Trend
if "pr_creationdate" in df.columns:
    pr_trend = (
        df.groupby("pr_creationdate")["pr_creationdate"]
        .count()
        .rename("pr_count")
        .reset_index()
        .sort_values("pr_creationdate")
    )
    plt.figure(figsize=(10,4))
    plt.plot(pr_trend["pr_creationdate"], pr_trend["pr_count"])
    plt.title("PR Volume Trend Over Time")
    plt.xlabel("PR Creation Date")
    plt.ylabel("Count of PRs")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# PO Volume Trend
if "po_createdon" in df.columns:
    po_trend = (
        df.groupby("po_createdon")["po_createdon"]
        .count()
        .rename("po_count")
        .reset_index()
        .sort_values("po_createdon")
    )
    plt.figure(figsize=(10,4))
    plt.plot(po_trend["po_createdon"], po_trend["po_count"])
    plt.title("PO Volume Trend Over Time")
    plt.xlabel("PO Creation Date")
    plt.ylabel("Count of POs")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# ------------------------------------------------------
# 8. VENDOR PERFORMANCE ANALYSIS
# ------------------------------------------------------
if "po_vendornumber" in df.columns:
    vendor_perf = (
        df.groupby("po_vendornumber")["pr_to_po_ageing"]
        .mean()
        .rename("avg_pr_to_po_ageing")
        .reset_index()
        .sort_values("avg_pr_to_po_ageing")
    )
    print("\nVendor performance (average PR→PO ageing):")
    display(vendor_perf.head(20))

# ------------------------------------------------------
# 9. MATERIAL GROUP PERFORMANCE
# ------------------------------------------------------
if "materialgroupdesc" in df.columns:
    material_perf = (
        df.groupby("materialgroupdesc")["pr_to_po_ageing"]
        .mean()
        .rename("avg_cycle")
        .reset_index()
        .sort_values("avg_cycle", ascending=False)
    )
    print("\nMaterial group performance (average PR→PO ageing):")
    display(material_perf.head(20))

# ------------------------------------------------------
# 10. SLA BREACH ANALYSIS
# ------------------------------------------------------
if "sla_breach_flag" in df.columns:
    sla_summary = df["sla_breach_flag"].value_counts(dropna=False).reset_index()
    sla_summary.columns = ["sla_breach_flag", "count"]
    print("\nSLA Breach Summary:")
    display(sla_summary)

if "pr_cycle_sla_breach_flag" in df.columns:
    print("\nPR cycle SLA Breach Summary:")
    display(df["pr_cycle_sla_breach_flag"].value_counts(dropna=False).to_frame("count"))

if "po_cycle_sla_breach_flag" in df.columns:
    print("\nPO cycle SLA Breach Summary:")
    display(df["po_cycle_sla_breach_flag"].value_counts(dropna=False).to_frame("count"))

# ------------------------------------------------------
# 11. RECORD TYPE BREAKDOWN
# ------------------------------------------------------
if "record_type" in df.columns:
    record_type_counts = df["record_type"].value_counts(dropna=False).reset_index()
    record_type_counts.columns = ["record_type", "count"]
    print("\nRecord Type Breakdown:")
    display(record_type_counts)

# ------------------------------------------------------
# 12. OUTLIER DETECTION (BOXPLOTS)
# ------------------------------------------------------
for col in numeric_cols:
    plt.figure(figsize=(6,4))
    sns.boxplot(x=df[col].dropna())
    plt.title(f"Boxplot for {col} (Outlier Check)")
    plt.xlabel(col)
    plt.show()

# ------------------------------------------------------
# 13. BIAS / ETHICS ANALYSIS
# ------------------------------------------------------
# Check ageing by PR company code
if "pr_companycode" in df.columns:
    pr_cc_delay = (
        df.groupby("pr_companycode")["pr_to_po_ageing"]
        .mean()
        .rename("avg_delay")
        .reset_index()
        .sort_values("avg_delay", ascending=False)
    )
    print("\nAverage PR→PO ageing by PR company code:")
    display(pr_cc_delay)

# Check vendor country bias
if "po_countrykey" in df.columns:
    country_delay = (
        df.groupby("po_countrykey")["pr_to_po_ageing"]
        .mean()
        .rename("avg_delay")
        .reset_index()
        .sort_values("avg_delay", ascending=False)
    )
    print("\nAverage PR→PO ageing by vendor country:")
    display(country_delay)

print("\nEDA Complete.")
